In [ ]:
# 1. Instalar o uv para gerenciar dependências
!pip install uv --quiet

# 2. Usa o uv para resolver e instalar o PYield respeitando o ambiente do Colab
!uv pip install PYield

# Importar a biblioteca PYield
import pyield as yd
from pyield import bday

print("PYield version:", yd.__version__)

In [ ]:
# Importar a biblioteca PYield
import pyield as yd

yd.__version__

In [ ]:
# 1. Contagem de dias úteis
# A função `bday.count` calcula o número de dias úteis entre duas datas usando
# o calendário de dias úteis do Brasil. Formatos aceitos: `DD-MM-YYYY` ou `YYYY-MM-DD`.
du = bday.count("02-01-2024", "09-01-2024")
print("Dias úteis:", du)
# Lista de vários períodos (retorna Series Polars)
bday.count("01-01-2024", ["08-01-2024", "22-01-2024"])

In [ ]:
# 2. Offset de datas para cair em dias úteis.
# `bday.offset(data, n)` ajusta para o próximo dia útil considerando feriados.
ajustada = bday.offset("20-11-2024", 0)  # 20/11/2024 é feriado nacional recente
print("Data ajustada:", ajustada)

In [ ]:
# 2. A contagem de dias úteis e de offset são funções vetorizadas do Polars.
# Aceitam listas ou Series do Polars ou Pandas.
# Mas sempre retornam Series do Polars.
datas = ["15-11-2023", "20-11-2024", "25-12-2023"]
offsets = [0, 0, 2]
bday.offset(datas, offsets)

In [ ]:
# Vamos ajustar os vencimentos das LTN para coindirem com dias úteis.
# Primeiro, obtemos os dados das LTN. Vamos manter somente as colunas necessárias.
df_ltn = yd.ltn.data(date="27-11-2025").select(
    data_ref="ReferenceDate",
    titulo="BondType",
    vencimento="MaturityDate",
    taxa="IndicativeRate",
)
df_ltn

In [ ]:
# Agora, aplicamos o offset para ajustar os vencimentos.
vencimentos_ajustados = bday.offset(df_ltn["vencimento"], 0)
df_ltn = df_ltn.with_columns(
    vencimentos_ajustados.alias("vencimento_ajustado"),
)
df_ltn

In [ ]:
# Vamos calcular o DU entre a data de referência e o vencimento.
# Tanto faz usar o vencimento original ou o ajustado, pois a contagem só considera DU.
du_vencimento = bday.count(df_ltn["data_ref"], df_ltn["vencimento"])
du_vencimento_ajustado = bday.count(df_ltn["data_ref"], df_ltn["vencimento_ajustado"])
df_ltn = df_ltn.with_columns(
    du_vencimento.alias("du"),
    du_vencimento_ajustado.alias("du_ajustado"),
)
df_ltn

In [ ]:
# Podemos calcular a taxa forward entre os vértices das LTNs dado que essas taxas já
# são spot (as LTN não pagam cupom).
# Esse cálculo é também vetorizado, aceitando Series do Polars ou Pandas.
tx_forward = yd.forwards(bdays=df_ltn["du"], rates=df_ltn["taxa"])
df_ltn = df_ltn.with_columns(
    tx_forward.alias("taxa_fwd"),
)
df_ltn


In [35]:
# 2. Futuros DI (B3)
# Obtém dados de contratos futuros de DI1 (taxas de vencimento).
# Faz requisição à B3, portanto é necessário acesso à internet.
df = yd.futures(contract_code="DI1", date="26-12-2023")
df

TradeDate,TickerSymbol,ExpirationDate,BDaysToExp,DaysToExp,OpenContracts,TradeCount,TradeVolume,FinancialVolume,DV01,SettlementPrice,OpenRate,MinRate,AvgRate,MaxRate,CloseAskRate,CloseBidRate,CloseRate,SettlementRate,ForwardRate
date,str,date,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-12-26,"""DI1F24""",2024-01-02,4,7,7647468,2174,134773,13453757574,0.141927,99825.32,0.1163,0.1163,0.11644,0.11652,0.11642,0.11646,0.11642,0.11644,0.11644
2023-12-26,"""DI1G24""",2024-02-01,26,37,544319,1418,96584,9549190235,0.913624,98869.28,0.11654,0.1165,0.11652,0.11654,0.11648,0.11652,0.1165,0.11652,0.116535
2023-12-26,"""DI1H24""",2024-03-01,45,66,480846,45,3334,327016888,1.57177,98085.0,0.1143,0.1143,0.11433,0.11442,0.11432,0.1144,0.1144,0.11436,0.111411
2023-12-26,"""DI1J24""",2024-04-01,65,97,3032933,380,65235,6345841593,2.254368,97276.33,0.11305,0.11295,0.11298,0.11305,0.1129,0.113,0.113,0.113,0.109946
2023-12-26,"""DI1K24""",2024-05-02,87,128,260810,2,315,30372021,2.995359,96421.36,0.1115,0.1114,0.1114,0.1115,null,0.1114,0.1114,0.11133,0.106411
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-12-26,"""DI1F34""",2034-01-02,2513,3660,11036,127,293,10998664,33.945329,37552.8,0.1033,0.103,0.10324,0.1033,0.1029,0.1039,0.1033,0.1032,0.102209
2023-12-26,"""DI1F35""",2035-01-02,2761,4025,10940,0,0,0,33.771251,34011.83,null,null,null,null,0.0999,null,null,0.10344,0.105875
2023-12-26,"""DI1F36""",2036-01-02,3010,4390,22054,0,0,0,33.330328,30796.2,null,null,null,null,null,null,null,0.10363,0.105739


In [ ]:
# Vamos usar as taxas de ajuste do DI1 como base na geração de taxas interpoladas.
df_di1 = df.select(
    data_ref="TradeDate",
    vencimento="ExpirationDate",
    taxa_ajuste="SettlementRate",
    du="BDaysToExp",
)
# Classe de interpolação da PYield
flat_fwd = yd.Interpolator(
    method="flat_forward",
    known_bdays=df_di1["du"],
    known_rates=df_di1["taxa_ajuste"],
)
flat_fwd

shape: (38, 2)
┌──────┬─────────┐
│ bday ┆ rate    │
│ ---  ┆ ---     │
│ i64  ┆ f64     │
╞══════╪═════════╡
│ 4    ┆ 0.11644 │
│ 26   ┆ 0.11652 │
│ 45   ┆ 0.11436 │
│ 65   ┆ 0.113   │
│ 87   ┆ 0.11133 │
│ …    ┆ …       │
│ 2513 ┆ 0.1032  │
│ 2761 ┆ 0.10344 │
│ 3010 ┆ 0.10363 │
│ 3263 ┆ 0.10382 │
│ 3512 ┆ 0.10382 │
└──────┴─────────┘

In [42]:
# Podemos agora calcular taxas interpoladas para qualquer DU.
# Se du é conhecido, retorna a taxa existente.
# Se du não é conhecido, retorna a taxa interpolada (flat forward no caso).
du_alvo = 4
print("DU existente  :", flat_fwd.interpolate(du_alvo))  # DU existente
du_alvo = 10
print("DU interpolado:", flat_fwd.interpolate(du_alvo))  # DU interpolado

DU existente  : 0.11644
DU interpolado: 0.11649672668149402


In [ ]:
# Converter para Pandas (se desejar interoperar com outras libs):
df_pandas = df.to_pandas(use_pyarrow_extension_array=True)
df_pandas

## 3. Próximos Passos
- Explorar módulos de títulos públicos (`pyield.tn`) e índices (`pyield.ipca`).
- Integrar com análises de curva usando `pyield.interpolator`.

Consulte a documentação para mais detalhes.